## Lasso Regression

The explaination of this algorithm was taken from the book *The Elements of Statistical Learning*, chapter 3.

## Importing packages

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from os.path import join
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin

## Explaining the Algorithm

Lasso Regression shrinks the regression coefficients by imposing a penalty on their size. The ridge coefficients minimize a penalized residual sum of squares, given by the following equation:

$$ \hat{\beta}^{lasso} = \argmin_{\beta} \left\{ \frac{1}{2} \sum_{i=1}^{N} \left( y_i - \beta_0 - \sum_{j=1}^{p} x_{ij} \beta_j \right)^2 + \lambda \sum_{j=1}^{p}\left| \beta_j \right| \right\} $$

It can also be wrote in the matrix form:

$$ RSS(\lambda) = (\bold{y} - \bold{X} \beta)^T (\bold{y} - \bold{X}  \beta) + \lambda \beta^T \beta$$

Due to the constraint $\left| \beta_j \right|$ the solutions are nonlinear in the $y_i$, and there is no closed form expression as in ridge regression. The  Lasso algorithm is a quadratic programming problem, so we can use some famous optimizations algorithms such as Gradient Descent and Coordinate Descent. In this code I will implement a Coordinate Descent Algorthm to compute the Lasso Solution. For a more detailed explanaition of the algorithm. See the references. [[1]](https://stats.stackexchange.com/questions/347796/coordinate-descent-for-lasso), [[2]](https://www.stat.cmu.edu/~ryantibs/convexopt/lectures/coord-desc.pdf), [[3]](https://arxiv.org/pdf/0803.3876).


### Mathematical Steps in Coordinate Descent
1. Start with initial coefficients: Initialize $\beta=0$

2. Iterative updates: For each coefficient $\beta_j$
    - Compute the partial residual:
        $$r_j = y - \sum_{k \ne j} X_k\beta_k$$
    - Compute the "soft-thresholded" update:
        $$\beta_j = soft-threshold\left( \frac{X_j^{\intercal}r_j}{n}, \frac{\lambda}{n} \right)$$
    where the soft-thresholding operator is defined as:
        $$soft-threshold(z,\gamma) = \begin{cases}
   z-\gamma &\text{if } z \gt \gamma \\
   z+\gamma &\text{if } z \lt -\gamma \\
   0 &\text{if } \left| z \right| \le \gamma
\end{cases} $$

3. Repeat: Iterate over all coefficients until convergence, i.e., until the updates become smaller than a tolerance threshold.

There is another extra step for updating the Intercept. For the model to fit well. The residuals should have a mean of zero $(mean(r) = 0)$. This ensures that the predictions are unbiased, i.e., the average prediction matches the average observed value of $y$.

#### Intercept Update Formula
The intercept is updated to realign the residuals by minimizing their mean:
 $$\beta_0 = mean(y) - mean(X\beta)$$

In [2]:
def soft_threshold(z, gamma):
    if z > gamma:
        return z - gamma
    elif z < gamma:
        return z + gamma
    elif abs(z) < gamma:
        return 0

In [3]:
alpha = 0.5
max_iter = 10000
X = np.array([[1], [2], [3], [4]])
y = np.array([1, 3, 2, 5])

# Centering Y for not having to calculate the intercept
y_mean = np.mean(y)
y_centered = y-y_mean

In [4]:
N, p = X.shape
weights = np.zeros(p)

for i in range(max_iter):
    for j in range(p):
        r_j = y_centered - X @ weights + X[:, j] * weights[j]
        rho_j = X[:, j].T @ r_j
        weights[j] = soft_threshold(rho_j/N, alpha/N)

weights 

array([1.25])

In [5]:
b_0 = y_mean - np.mean(X @ weights)


In [6]:
np.mean([5,6,7,8])

6.5

## Custom Model

In [7]:
class LassoRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=0.5, max_iter=10000, tol=0.01):
        self.alpha = alpha
        self.max_iter = max_iter
        self.tol = tol

    def fit(self, X, y):
        # Centering Y for not having to calculate the intercept
        self.y = y.copy()
        self.X = X.copy()

        y_mean = np.mean(y)
        self.y = self.y-y_mean

        self.N, self.p = self.X.shape
        self.weights = np.zeros(self.p)
        prev_weights = np.ones(self.p)

        for i in range(self.max_iter):
            if np.linalg.norm(self.weights - prev_weights) < self.tol:
                break

            # Update previous weights
            prev_weights = self.weights

            for j in range(self.p):
                # Calculate the residuals
                r_j = self.y - self.X @ self.weights + self.X[:, j] * self.weights[j]
                rho_j = self.X[:, j].T @ r_j
                self.weights[j] = self._soft_threshold(rho_j/self.N, self.alpha/self.N)
        
        
        self.b_0 = y_mean - np.mean(X @ self.weights)

        return self

    def predict(self, X):
        return X @ self.weights + self.b_0
    
    def _soft_threshold(self, z, gamma):
        if z > gamma:
            return z - gamma
        elif z < gamma:
            return z + gamma
        elif abs(z) < gamma:
            return 0
        
    def get_variance(self):
        y_hat = self.predict(self.X)
        return 1/(self.N - self.p - 1) * np.sum((y_hat - self.y)**2)
    
    def get_params_covariance(self) -> np.ndarray:
        """The variance–covariance matrix of the least squares parameter estimates

        Returns:
            np.ndarray: The variance–covariance matrix
        """
        return np.linalg.inv(self.X.T @ self.X) * self.get_variance()

In [8]:
X = np.array([[0,0], [1, 1], [2, 2]])
y = np.array([0, 1, 2])

In [25]:
my_lasso = LassoRegressor(alpha=0.1).fit(X, y)
my_y_hat = my_lasso.predict(X)
my_y_hat

array([0.72222222, 1.        , 1.27777778])

In [26]:
my_lasso.weights, my_lasso.b_0

(array([ 0.63333333, -0.35555556]), 0.7222222222222222)

In [27]:
from sklearn.linear_model import Lasso

In [28]:
sk_lasso = Lasso(alpha=0.1).fit(X, y)
sk_y_hat = sk_lasso.predict(X)
sk_y_hat

array([0.15, 1.  , 1.85])

In [29]:
sk_lasso.coef_, sk_lasso.intercept_

(array([0.85, 0.  ]), 0.15000000000000002)

In [30]:
np.mean(sk_y_hat)

1.0

In [31]:
np.mean(my_y_hat)

1.0

In [15]:
# Lasso Regression 
class LassoRegression(): 
	def __init__(self, learning_rate, iterations, l1_penalty): 
		self.learning_rate = learning_rate 
		self.iterations = iterations 
		self.l1_penalty = l1_penalty 

	# Function for model training 
	def fit(self, X, Y): 
		# no_of_training_examples, no_of_features 
		self.m, self.n = X.shape 
		# weight initialization 
		self.W = np.zeros(self.n) 
		self.b = 0
		self.X = X 
		self.Y = Y 
		# gradient descent learning 
		for i in range(self.iterations): 
			self.update_weights() 
		return self

	# Helper function to update weights in gradient descent 
	def update_weights(self): 
		Y_pred = self.predict(self.X) 
		# calculate gradients 
		dW = np.zeros(self.n) 
		for j in range(self.n): 
			if self.W[j] > 0: 
				dW[j] = (-2 * (self.X[:, j]).dot(self.Y - Y_pred) +
						self.l1_penalty) / self.m 
			else: 
				dW[j] = (-2 * (self.X[:, j]).dot(self.Y - Y_pred) -
						self.l1_penalty) / self.m 

		db = -2 * np.sum(self.Y - Y_pred) / self.m 

		# update weights 
		self.W = self.W - self.learning_rate * dW 
		self.b = self.b - self.learning_rate * db 
		return self

	# Hypothetical function h(x) 
	def predict(self, X): 
		return X.dot(self.W) + self.b 


In [35]:
gd_lasso = LassoRegression(learning_rate=0.1, iterations=1000, l1_penalty=0.1).fit(X, y)
gd_y_hat = gd_lasso.predict(X)
gd_y_hat

array([0.025, 1.   , 1.975])

In [36]:
np.mean(gd_y_hat)

1.0